In [1]:
import sys
sys.path.append("/Users/mocquin/MYLIB10/MODULES/noise3D/")

In [2]:
import numpy as np

import noise3d

# We'll be creating 3D sequences with T=100 frames, V=100 lines, and H=100 columns.
T = 100
V = 100
H = 100
# but you can define any shape

# Create some seq to apply operators on
seq = noise3d.genseq.genseq_3dnoise_seq(T, V, H, [1 for i in range(7)], [0 for i in range(7)])

# Basic noise variances

In [15]:
# Getting the individual noise variances 
print(noise3d.noise.var_nt(seq))
print(noise3d.noise.var_nv(seq))
print(noise3d.noise.var_nh(seq))
print(noise3d.noise.var_ntv(seq))
print(noise3d.noise.var_nth(seq))
print(noise3d.noise.var_nvh(seq))
print(noise3d.noise.var_ntvh(seq))
print()

# Wrapping all in a function, as well as total variance
noise_vars = noise3d.noise.get_all_3D_noise_var(seq, names=True)
for var, name in zip(noise_vars[:-1], noise_vars[-1]):
    print(f"Var({name:3}) : {var:.2f}")



1.1050255666161428
0.9410157285292603
1.1284436757089504
0.9666840215599901
0.9540686888098918
1.0049240568167281
0.9709206839194532

Var(t  ) : 1.11
Var(v  ) : 0.94
Var(h  ) : 1.13
Var(tv ) : 0.97
Var(th ) : 0.95
Var(vh ) : 1.00
Var(tvh) : 0.97
Var(tot) : 7.04


## Faster equivalent version

A faster equivalent version 

In [16]:
noise_vars = noise3d.noise.get_all_3d_noise_var_fast(seq, names=True)
for var, name in zip(noise_vars[:-1], noise_vars[-1]):
    print(f"Var({name:3}) : {var:.2f}")


Var(t  ) : 1.11
Var(v  ) : 0.94
Var(h  ) : 1.13
Var(tv ) : 0.97
Var(th ) : 0.95
Var(vh ) : 1.00
Var(tvh) : 0.94
Var(tot) : 7.04


In [ ]:
%timeit noise3d.noise.get_all_3d_noise_var_fast(seq)
%timeit noise3d.noise.get_all_3d_noise_var(seq)